

### Imports



In [ ]:
from PIL import Image, ImageFont, ImageDraw
import numpy
from pylab import *

import random
import os
import string
import json
import csv
import cv2
from google.colab.patches import cv2_imshow

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).





### Fill the docs with data

In [ ]:

 # There should be 1 image in background folder 
background_path = "/content/drive/MyDrive/p2-Data/backgrounds"  # images path
font_path = "/content/drive/MyDrive/p2-Data/fonts"   # Font path
output_path = "/content/drive/MyDrive/p2-Data/test/pakistan" # output file
num_images = 3000
show_bbox = False

font_color_pak = '#abc79b'
font_color_canada = '#6695bb'
font_color_china = '#c8d5df'
font_color_queensland = '#608814'
font_color_usa = '#68574d'



def fill_docs_with_data(text_font, c_font, data, info) :
	card_title = ' ' #str_title
	positions = [] #pos
	margin_top = 10
	margin_bottom = 30
	
	color = (0,0,0) 
	size = text_font.getsize(card_title) #sz
	x, y = (info[0]-size[0])/2, margin_top
	draw.text((x,y), card_title, font_color_pak, font=text_font)
	
	Dy = (info[1]-margin_top-margin_bottom-size[1]-30) / 4
	Dy = 50 
	y = margin_top + size[1] + 110 #margins = 150 usa, 110 queensland,canada,pak, 200 china

	for key, val in data.items() :
		x = 10
		size_key = text_font.getsize(key)
		x = (info[0]-size[0])/2-180 
		draw.text((x,y), key + ":", color, font=text_font)
		
		x += size_key[0]+10
		str = val
		options = 11
		cond = 10
		if options >= cond :
			cont = random.randint(0, len(str)-1)
			post = str[(cont+1):]
			draw.text((x,y), str[:cont], color, font=text_font)
			wpre, h = text_font.getsize(str[:cont])
			if len(str) > cont+1 :
				w, h = text_font.getsize(str)
				wpost, h = text_font.getsize(post)
				draw.text((x+w-wpost,y), str[c+1:], color, font=text_font)
				
			dx = random.randint(-2,2)
			dy = random.randint(-2,2)
			while dx == 0 and dy == 0 :
				dx = random.randint(-2,2)
				dy = random.randint(-2,2)
			xc, yc = x+wpre+dx, y+dy
			draw.text((xc, yc), str[cont], color, font=text_font)
			wc, hc = text_font.getsize(str[cont])
			c_off = text_font.getoffset(str[cont])
			if xc+c_off[0]>=0 and yc+c_off[1]>=0 and xc+wc<info[0] and yc+hc<info[1] :
				positions.append([xc+c_off[0], yc+c_off[1], xc+wc, yc+hc])
				if show_bbox :
					draw.rectangle(positions[-1], outline = 'blue')
		else :
			draw.text((x,y), str, color, font=text_font)
		y += Dy
			
	return positions


fonts_tyles = os.listdir(font_path)
backgrounds = os.listdir(background_path)
csv_file = open('/content/drive/MyDrive/p2-Data/DATA.csv','r')


def print_docs(fonts_tyles, backgrounds, csv_file):
	cards = csv.DictReader(csv_file)
 	persons = []
	# save cards

	for person in cards :
		persons.append(person)


	for i in range(1,len(persons),1) : 
		if i >=num_images : 
			break
		if i % 1000 == 0 :
			print("i = {}".format(i))
		# Can change font style
		fontstyle = os.path.join(font_path, fonts_tyles[3]) 
		
		fontsize = 30 # random.randint(20,30)#
		font = ImageFont.truetype(fontstyle, fontsize)
		c_fontstyle = fontstyle
		c_fontsize = fontsize + random.randint(-2,2)
		c_font = ImageFont.truetype(c_fontstyle, c_fontsize)
		bg = os.path.join(background_path, backgrounds[random.randint(0,len(backgrounds)-1)])
		print(bg)
		if not '.jpg' in bg:
			continue
		im = Image.open(bg)
		im_info = im.size
		print(im_info)
		draw = ImageDraw.Draw(im)
		
		pos = fill_docs_with_data(font, c_font, dict(persons[i]), im_info)
		
		file_name = "passport.{}.jpg".format(i)
		
		im.save(os.path.join(output_path, file_name),quality=95)


	

print_docs()

### Apply data augmentation



In [ ]:
import cv2 as cv
import json 
import os
import numpy as np
import cv2

def data_aug(image, techique):
  if techique == "rotation":
    angle = 5
    
    # grab the dimensions of the image and then determine the
    # centre
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = cv2.warpAffine(image, M, (nW, nH))

    #    image = cv2.resize(image, (w,h))
    return image

    elif technique == "shearing":
      shear_factor = 0.1
      shear_factor = random.uniform(shear_factor,0.2)

      w,h = image.shape[1], image.shape[0]


      M = np.array([[1, abs(shear_factor), 0],[0,1,0]])

      nW =  image.shape[1] + abs(shear_factor*image.shape[0])


      image = cv2.warpAffine(image, M, (int(nW), image.shape[0]))


      image = cv2.resize(image, (w,h))

      scale_factor_x = nW / w


      return image


# save augmented docs

def save_tempered_docs(path):
  # Opening JSON file 
  for j in ["Pakistan"]: #,"Canada","QueensLand","USA"
    
    f = open(path+'/annotation.json',) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 

    # Iterating through the json 
    # list
    count= 0
    for img_name in os.listdir(path):
      
        if count == 1500:
          break
        print("Processing : ", j, "  ", count , "/ ", 1500)
        count+=1
        if not ".jpg" in img_name:
            continue
        im=cv.imread(path+"/"+img_name)
        ls = { i["path"]: i["bbox"]  for i  in data }
        f.close() 
        for i in ls[img_name]:
            x_min = int(i[1])
            y_min = int(i[0])
            x_max = int(i[3])
            y_max = int(i[2])
            im[ x_min:x_max, y_min:y_max]= im[ x_min:x_max, y_min:y_max]+np.random.randint(-100,100)  # Adding noise
            cv2_imshow(im)
            

            if count % 2 ==0:
              # rotation
              im = data_aug(im, "rotation")
            elif count %2 !=0:
              # shearing
              im = data_aug(im, "shearing")
            
            
            
        # Save randomly tampered card
        cv.imwrite(path+"/noised/"+img_name,im) 
    f.close()


save_augmented_cards()
    

